In [1]:
import pandas as pd
import os

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
df_t = pd.read_parquet("/Users/gufran/Desktop/PfsPredictionLungCancer/radiology/radiomics_features/LUNG_RADIOMICS_spacing1.0_MirpOn_Window1350.250_allImageTypes_bw20.parquet", engine='pyarrow').reset_index()
df_v = pd.read_parquet("/Users/gufran/Desktop/PfsPredictionLungCancer/radiology/radiomics_features/LUNG_RADIOMICS_spacing1.0_MirpOn_Window1350.250_allImageTypes_bw20_VALIDATION.parquet", engine='pyarrow').reset_index()
df_t = df_t.rename(columns={'main_index': 'dmp_pt_id'})
df_v = df_v.rename(columns={'main_index': 'radiology_accession_number'})

df_t.drop(["job_tag"], axis=1, inplace=True)
df_v.drop(["job_tag"], axis=1, inplace=True)

df_v['radiology_accession_number'] = pd.to_numeric(df_v['radiology_accession_number'], errors='coerce').astype(object)

df_t.shape, df_v.shape

((3996, 1690), (1176, 1690))

In [3]:
df_t.head()

,dmp_pt_id,lesion_index,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,...,wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,wavelet-LLL_glszm_ZoneEntropy,wavelet-LLL_glszm_ZonePercentage,wavelet-LLL_glszm_ZoneVariance,wavelet-LLL_ngtdm_Busyness,wavelet-LLL_ngtdm_Coarseness,wavelet-LLL_ngtdm_Complexity,wavelet-LLL_ngtdm_Contrast,wavelet-LLL_ngtdm_Strength
0,P-0000239,1,0.609991,0.547450,17.435125,31.847914,33.970576,33.105891,25.553865,39.962482,...,1046.802692,0.001751,6.371523,0.158582,3270.745421,0.128236,0.002030,1733.014821,0.026899,5.484464
1,P-0000239,1,0.602659,0.557320,17.530776,31.455496,32.557641,32.695565,24.351591,39.089641,...,646.730034,0.003363,6.101283,0.159459,3298.620481,0.168471,0.002173,1198.254380,0.026713,4.042621
2,P-0000239,1,0.626577,0.535450,17.343918,32.391268,37.802116,38.897301,26.400758,41.448764,...,904.065174,0.003693,6.650200,0.151809,3167.716176,0.109002,0.002366,1716.921780,0.033027,7.627719
3,P-0000239,1,0.685398,0.640327,18.342757,28.645906,34.132096,38.897301,28.844410,39.344631,...,947.556576,0.001619,6.511064,0.108050,4967.802504,0.113148,0.002493,1183.899751,0.015043,7.936936
4,P-0000239,1,0.633165,0.559689,17.613374,31.469915,34.132096,38.897301,28.844410,39.344631,...,2475.982809,0.001237,6.611130,0.177028,2702.966436,0.061176,0.002091,3721.568696,0.033197,14.876286


In [4]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3996 entries, 0 to 3995
Columns: 1690 entries, dmp_pt_id to wavelet-LLL_ngtdm_Strength
dtypes: float64(1688), int64(1), object(1)
memory usage: 51.5+ MB


In [5]:
def process_clinical(df, selected_columns=None, correlation_threshold = 0.2, drop_na = True):
    object_float_columns = ["pack_years","halo_tumor_quality"]
    good_object_columns = ["histo","pdl1_tiss_site"]
    
    remove_columns = list(df.select_dtypes(include=['object']).columns)
    # if "dmp_pt_id" in remove_columns: remove_columns.append("dmp_pt_id")
    if "dmp_pt_id" in remove_columns: remove_columns.remove("dmp_pt_id")
    if "pdl1_image_id" in remove_columns: remove_columns.remove("pdl1_image_id")
    
    remove_columns.append("record_id")
    remove_columns.append("pfs_censor")
    # remove_columns.append("radiology_accession_number")
    remove_columns.append("os_int")
    remove_columns.append("bor")
    remove_columns.append("did_acc")
    remove_columns.append("sex")
    remove_columns.append("deid")
    remove_columns.append("label")
    remove_columns.append("clinical_pdl1_score") #removing because of empty cells in test data
    remove_columns.append("js_pdl1_score") #removing because of empty cells in test data
    
    for c in object_float_columns:
        remove_columns.remove(c)
        
    df = df.drop(columns=remove_columns)
    
    for c in object_float_columns:
        df[c] = pd.to_numeric(df[c], errors='coerce').astype(float)
    df["hist_adeno"] = df["hist_adeno"].astype(int)
    
    if selected_columns is None:
        correlations = df.drop(["dmp_pt_id" if "dmp_pt_id" in df.columns else "radiology_accession_number"], axis=1).iloc[:, :-1].corrwith(df['pfs'])
        selected_columns = correlations[(correlations >= correlation_threshold) | (correlations <= -correlation_threshold)].index
        selected_columns = list(selected_columns)
        if "dmp_pt_id" in df.columns: selected_columns.append("dmp_pt_id")
        if "radiology_accession_number" in df.columns: selected_columns.append("radiology_accession_number")
    
    if "dmp_pt_id" not in df.columns: 
        selected_columns.remove("dmp_pt_id")
        selected_columns.append("radiology_accession_number")
        
    for sc in selected_columns:
        if sc in ["dmp_pt_id", "radiology_accession_number","pfs"]: continue
        print(f"Correlation with {sc}: {df['pfs'].corr(df[sc])}")
    
    df = df[selected_columns]
    if drop_na: df = df.dropna().reset_index(drop=True)
    
    return df, selected_columns

In [6]:
df_clinical = pd.read_csv('/Users/gufran/Desktop/PfsPredictionLungCancer/clinical_data/clinical_data.csv').drop(["pdl1_image_id"], axis=1)
print(df_clinical.shape)

df_clinical_tr = df_clinical.drop(["radiology_accession_number"], axis=1)
df_clinical_te = df_clinical.drop(["dmp_pt_id"], axis=1)

df_clinical_tr, selected_columns = process_clinical(df_clinical_tr, correlation_threshold = 0.11)
df_clinical_te, _ = process_clinical(df_clinical_te, correlation_threshold = 0.11, selected_columns=selected_columns)

(366, 52)
Correlation with albumin: 0.26071203044553243
Correlation with dnlr: -0.11717087983081641
Correlation with ecog: -0.1665222793734818
Correlation with albumin: 0.26071203044553243
Correlation with dnlr: -0.11717087983081641
Correlation with ecog: -0.1665222793734818


In [7]:
df_t = pd.merge(df_t, df_clinical_tr, on='dmp_pt_id', how='inner')
df_v = pd.merge(df_v, df_clinical_te, on='radiology_accession_number', how='inner')

df_t.shape, df_v.shape

((3996, 1694), (1176, 1694))

In [8]:
correlation_threshold = 0.1
correlations = df_t.drop("dmp_pt_id", axis=1).corrwith(df_t['pfs'])
selected_columns = correlations[(correlations >= correlation_threshold) | (correlations <= -correlation_threshold)].index

selected_columns = list(selected_columns)
# selected_columns.append("albumin")
# selected_columns.append("ecog")

In [9]:
for sc in selected_columns:
    if sc in ["dmp_pt_id", "pfs"]: continue
    print(f"Correlation with {sc}: {df_t['pfs'].corr(df_t[sc])}")

Correlation with original_firstorder_10Percentile: -0.16043499145721676
Correlation with original_firstorder_90Percentile: -0.20932506044661645
Correlation with original_firstorder_InterquartileRange: 0.10194727849112364
Correlation with original_firstorder_Mean: -0.20189748064173424
Correlation with original_firstorder_Median: -0.21681436649049524
Correlation with original_firstorder_Minimum: -0.10321323734142564
Correlation with original_firstorder_RootMeanSquared: 0.21326912797489453
Correlation with original_glcm_ClusterTendency: 0.10816573011428601
Correlation with original_glcm_Contrast: 0.11896090758668132
Correlation with original_glcm_DifferenceAverage: 0.10069171276473073
Correlation with original_glcm_InverseVariance: -0.13248411572515584
Correlation with original_glcm_SumSquares: 0.11097330448510836
Correlation with original_gldm_DependenceNonUniformityNormalized: 0.12216061095395497
Correlation with original_ngtdm_Complexity: 0.13049139631450526
Correlation with exponentia

In [12]:
df_v

,radiology_accession_number,lesion_index,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,...,wavelet-LLL_glszm_ZoneVariance,wavelet-LLL_ngtdm_Busyness,wavelet-LLL_ngtdm_Coarseness,wavelet-LLL_ngtdm_Complexity,wavelet-LLL_ngtdm_Contrast,wavelet-LLL_ngtdm_Strength,albumin,dnlr,pfs,ecog
0,190310,1,0.773650,0.497674,7.566988,15.204695,14.560220,15.811388,14.866069,17.000000,...,1.957267,0.005179,0.018377,27138.212449,0.718451,89.793935,3.9,2.5,26.1,1
1,190310,1,0.759573,0.487954,7.103445,14.557626,13.601471,15.000000,14.560220,16.552945,...,1.731737,0.007126,0.020969,16761.257501,0.616416,71.385709,3.9,2.5,26.1,1
2,190310,1,0.792993,0.468580,5.898571,12.588187,13.152946,12.083046,13.601471,14.628739,...,0.955822,0.008503,0.026378,11716.812512,0.602118,63.227113,3.9,2.5,26.1,1
3,190310,1,0.754851,0.549451,7.673298,13.965401,14.035669,15.000000,14.866069,17.117243,...,0.731963,0.015744,0.018415,15458.951270,0.791238,44.017116,3.9,2.5,26.1,1
4,190310,1,0.737045,0.463315,7.322990,15.805635,13.152946,21.400935,15.811388,21.494185,...,0.758003,0.012408,0.018273,13859.448289,0.660879,45.112721,3.9,2.5,26.1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,191182,2,0.337839,0.307031,9.498734,30.937349,27.459060,33.241540,13.928388,33.376639,...,24.889789,0.034983,0.006381,7593.595023,0.346688,14.225450,3.8,3.0,2.1,1
1172,191182,2,0.378958,0.300726,8.273343,27.511229,26.172505,29.120440,14.212670,29.189039,...,38.865663,0.032658,0.007893,4886.665975,0.162361,17.558279,3.8,3.0,2.1,1
1173,191182,2,0.345684,0.274051,8.166433,29.798920,26.400758,29.732137,13.892444,34.205263,...,35.637054,0.053079,0.007211,3238.598946,0.169399,9.309697,3.8,3.0,2.1,1
1174,191182,2,0.341878,0.295010,9.663671,32.757136,29.154759,32.572995,14.560220,35.958309,...,22.855035,0.020794,0.005950,11931.027675,0.236597,24.237781,3.8,3.0,2.1,1


In [14]:
df_t_t = df_t.groupby('dmp_pt_id').transform('mean')

In [16]:
df_t

,dmp_pt_id,lesion_index,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,...,wavelet-LLL_glszm_ZoneVariance,wavelet-LLL_ngtdm_Busyness,wavelet-LLL_ngtdm_Coarseness,wavelet-LLL_ngtdm_Complexity,wavelet-LLL_ngtdm_Contrast,wavelet-LLL_ngtdm_Strength,albumin,dnlr,pfs,ecog
0,P-0000239,1,0.609991,0.547450,17.435125,31.847914,33.970576,33.105891,25.553865,39.962482,...,3270.745421,0.128236,0.002030,1733.014821,0.026899,5.484464,3.0,3.3,1.8,1
1,P-0000239,1,0.602659,0.557320,17.530776,31.455496,32.557641,32.695565,24.351591,39.089641,...,3298.620481,0.168471,0.002173,1198.254380,0.026713,4.042621,3.0,3.3,1.8,1
2,P-0000239,1,0.626577,0.535450,17.343918,32.391268,37.802116,38.897301,26.400758,41.448764,...,3167.716176,0.109002,0.002366,1716.921780,0.033027,7.627719,3.0,3.3,1.8,1
3,P-0000239,1,0.685398,0.640327,18.342757,28.645906,34.132096,38.897301,28.844410,39.344631,...,4967.802504,0.113148,0.002493,1183.899751,0.015043,7.936936,3.0,3.3,1.8,1
4,P-0000239,1,0.633165,0.559689,17.613374,31.469915,34.132096,38.897301,28.844410,39.344631,...,2702.966436,0.061176,0.002091,3721.568696,0.033197,14.876286,3.0,3.3,1.8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3991,P-0045298,2,0.583836,0.428494,15.704461,36.650325,41.194660,41.194660,22.472205,41.388404,...,104.797049,0.059051,0.003033,5575.262403,0.128496,8.680779,3.6,1.9,12.7,1
3992,P-0045298,2,0.544379,0.369077,14.453937,39.162403,41.484937,41.194660,22.472205,43.139309,...,117.055375,0.060956,0.003595,4538.097910,0.126464,9.049781,3.6,1.9,12.7,1
3993,P-0045298,2,0.533278,0.378726,14.931061,39.424493,48.507731,42.190046,21.377558,48.672374,...,109.117767,0.068820,0.003392,4286.083430,0.139613,7.730850,3.6,1.9,12.7,1
3994,P-0045298,2,0.469694,0.395143,14.388747,36.414023,30.594117,34.481879,22.472205,34.568772,...,148.695555,0.071491,0.003768,3146.302617,0.083086,8.653283,3.6,1.9,12.7,1


In [15]:
df_t_t

,lesion_index,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,...,wavelet-LLL_glszm_ZoneVariance,wavelet-LLL_ngtdm_Busyness,wavelet-LLL_ngtdm_Coarseness,wavelet-LLL_ngtdm_Complexity,wavelet-LLL_ngtdm_Contrast,wavelet-LLL_ngtdm_Strength,albumin,dnlr,pfs,ecog
0,1.0,0.636928,0.576307,17.894897,31.14779,34.914166,37.039286,27.732208,40.358926,6936.281250,...,3179.635533,0.090819,0.002223,2760.583802,0.029861,12.090630,3.0,3.3,1.8,1.0
1,1.0,0.636928,0.576307,17.894897,31.14779,34.914166,37.039286,27.732208,40.358926,6936.281250,...,3179.635533,0.090819,0.002223,2760.583802,0.029861,12.090630,3.0,3.3,1.8,1.0
2,1.0,0.636928,0.576307,17.894897,31.14779,34.914166,37.039286,27.732208,40.358926,6936.281250,...,3179.635533,0.090819,0.002223,2760.583802,0.029861,12.090630,3.0,3.3,1.8,1.0
3,1.0,0.636928,0.576307,17.894897,31.14779,34.914166,37.039286,27.732208,40.358926,6936.281250,...,3179.635533,0.090819,0.002223,2760.583802,0.029861,12.090630,3.0,3.3,1.8,1.0
4,1.0,0.636928,0.576307,17.894897,31.14779,34.914166,37.039286,27.732208,40.358926,6936.281250,...,3179.635533,0.090819,0.002223,2760.583802,0.029861,12.090630,3.0,3.3,1.8,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3991,1.5,0.605400,0.443065,10.427869,25.43526,26.849856,25.489469,17.146605,28.450097,2509.326389,...,61.235191,0.041559,0.017948,3129.611258,0.170314,19.719824,3.6,1.9,12.7,1.0
3992,1.5,0.605400,0.443065,10.427869,25.43526,26.849856,25.489469,17.146605,28.450097,2509.326389,...,61.235191,0.041559,0.017948,3129.611258,0.170314,19.719824,3.6,1.9,12.7,1.0
3993,1.5,0.605400,0.443065,10.427869,25.43526,26.849856,25.489469,17.146605,28.450097,2509.326389,...,61.235191,0.041559,0.017948,3129.611258,0.170314,19.719824,3.6,1.9,12.7,1.0
3994,1.5,0.605400,0.443065,10.427869,25.43526,26.849856,25.489469,17.146605,28.450097,2509.326389,...,61.235191,0.041559,0.017948,3129.611258,0.170314,19.719824,3.6,1.9,12.7,1.0


In [64]:
# df_t.drop(["dmp_pt_id"], axis=1, inplace=True)
# df_v.drop(["radiology_accession_number"], axis=1, inplace=True)

df_t = df_t[selected_columns]
df_v = df_v[selected_columns]

In [65]:
X_train, y_train = df_t.drop(["pfs"], axis=1), df_t["pfs"]
X_test, y_test = df_v.drop(["pfs"], axis=1), df_v["pfs"]

In [66]:
X_test

,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_InterquartileRange,original_firstorder_Mean,original_firstorder_Median,original_firstorder_Minimum,original_firstorder_RootMeanSquared,original_glcm_ClusterTendency,original_glcm_Contrast,original_glcm_DifferenceAverage,...,wavelet-LLL_firstorder_Median,wavelet-LLL_firstorder_Minimum,wavelet-LLL_firstorder_RootMeanSquared,wavelet-LLL_glcm_ClusterTendency,wavelet-LLL_glcm_Contrast,wavelet-LLL_glcm_SumSquares,wavelet-LLL_ngtdm_Complexity,albumin,dnlr,ecog
0,-336.5,80.0,232.50,-97.708696,-55.0,-754.0,196.446405,192.102618,33.959419,4.078717,...,-184.746411,-2101.085613,561.711338,1563.901199,243.749234,451.912608,27138.212449,3.9,2.5,1
1,-296.0,72.0,211.00,-96.332293,-72.0,-618.0,175.732206,152.443723,23.305124,3.564688,...,-216.053009,-1751.408325,503.862395,1244.464875,179.356835,355.955428,16761.257501,3.9,2.5,1
2,-286.5,52.0,172.50,-89.674528,-65.5,-521.0,155.540188,118.043977,19.792805,3.464408,...,-190.663567,-1463.076294,442.451877,935.666670,154.331332,272.499500,11716.812512,3.9,2.5,1
3,-367.0,36.8,249.00,-158.234399,-145.0,-521.0,218.496377,167.143370,29.902218,4.237533,...,-430.964600,-1463.076294,621.569105,1323.460052,222.361430,386.455370,15458.951270,3.9,2.5,1
4,-323.0,44.0,221.50,-120.083578,-100.5,-521.0,185.142368,145.606646,26.090809,3.915008,...,-297.833115,-1463.076294,526.362145,1155.754413,193.101254,337.213917,13859.448289,3.9,2.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,-234.9,11.0,108.25,-79.152778,-47.0,-454.0,125.592702,64.099839,12.536466,2.369677,...,-136.525055,-1240.312500,358.640738,523.660995,96.179783,154.960194,7593.595023,3.8,3.0,1
1172,-124.0,17.0,69.00,-45.135057,-28.0,-374.0,80.888946,29.370230,6.305726,1.672006,...,-79.855190,-1075.861816,231.706467,240.814694,49.008330,72.455756,4886.665975,3.8,3.0,1
1173,-119.0,16.0,72.00,-45.069490,-33.0,-361.0,73.194464,21.599795,5.478303,1.619794,...,-93.494545,-999.430786,209.039023,176.409223,41.999565,54.602197,3238.598946,3.8,3.0,1
1174,-203.6,10.0,111.50,-80.642680,-54.0,-655.0,124.662078,62.205736,12.172869,2.357429,...,-156.578415,-1821.798584,352.787842,492.887134,91.695847,146.145745,11931.027675,3.8,3.0,1


In [67]:
classifiers_reg = {
    # 'SVM': {
    #     'name': 'Support Vector Machine',
    #     'classifier': SVR(),
    #     'param_grid': {'C': [0.1, 1.0, 10.0], 'kernel': ['linear', 'rbf']}
    # },
    'XGBoost': {
        'name': 'XGBoost',
        'classifier': XGBRegressor(),
        'param_grid': {'n_estimators': [50, 100, 200], 'max_depth': [3, 4, 5]}
    },
    'AdaBoost': {
        'name': 'AdaBoost',
        'classifier': AdaBoostRegressor(),
        'param_grid': {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.5, 1.0]}
    },
    'RandomForest': {
        'name': 'Random Forest',
        'classifier': RandomForestRegressor(),
        'param_grid': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}
    },
    'DecisionTree': {
        'name': 'Decision Tree',
        'classifier': DecisionTreeRegressor(),
        'param_grid': {'max_depth': [None, 10, 20]}
    }
}

In [68]:
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

best_models = {}

for clf_name, clf_info in classifiers_reg.items():
    print(f"Performing GridSearchCV for {clf_info['name']}...")
    
    clf = clf_info['classifier']
    param_grid = clf_info['param_grid']
    
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    best_models[clf_name] = grid_search.best_estimator_

print()

for clf_name, best_model in best_models.items():
    print(f"Evaluating {clf_name} on test data...")
    
    y_pred = best_model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    pearson_coefficient, _ = pearsonr(y_test, y_pred)
    
    print(f"Mean Squared Error on Test Data: {mse:.4f}")
    print(f"Pearson Correlation Coefficient on Test Data: {pearson_coefficient:.4f}\n")

Performing GridSearchCV for XGBoost...
Performing GridSearchCV for AdaBoost...
Performing GridSearchCV for Random Forest...
Performing GridSearchCV for Decision Tree...

Evaluating XGBoost on test data...
Mean Squared Error on Test Data: 172.3040
Pearson Correlation Coefficient on Test Data: 0.0246

Evaluating AdaBoost on test data...
Mean Squared Error on Test Data: 143.6442
Pearson Correlation Coefficient on Test Data: 0.2945

Evaluating RandomForest on test data...
Mean Squared Error on Test Data: 145.8118
Pearson Correlation Coefficient on Test Data: 0.2644

Evaluating DecisionTree on test data...
Mean Squared Error on Test Data: 170.9679
Pearson Correlation Coefficient on Test Data: 0.1767

